In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from os import listdir
import os.path
import numpy as np
import random
import json

img_dir= "/kaggle/input/amazonimage/IMAGEDATA/"
meta_dir = "/kaggle/input/amazon-bucket/METADATA/"

# getting whole metadata list 
def get_metadata(img_dir,meta_dir):
    metadata=[]
    n_images=0
#    img_list = listdir(img_dir)
#    N = len(img_list)
    N = 535234
    for i in range(N):
        if i%1000 == 0:
            print("get_metadata: processing (%d/%d)..." % (i,N))
        jpg_path = '%s%05d.jpg' % (img_dir,i+1)
        json_path = '%s%05d.json' % (meta_dir,i+1)
        if os.path.isfile(jpg_path) and os.path.isfile(json_path):
            d = json.loads(open(json_path).read())
            metadata.append(d)
            n_images = n_images + 1
        else:
            metadata.append({})
    print("get_metadata: Available Images: %d" % n_images)
    return metadata,n_images

# getting instance list
def get_instance_data(metadata):
    instances={}
    N = len(metadata)
    for i in range(N):
        if i%1000 == 0:
            print("get_instance_data: processing (%d/%d)..." % (i,N))
        if metadata[i]:
            quantity = metadata[i]['EXPECTED_QUANTITY']
            if quantity>0:
                bin_info = metadata[i]['BIN_FCSKU_DATA']
                bin_keys = list(bin_info.keys())
                for j in range(0,len(bin_info)):
                    instance_info = bin_info[bin_keys[j]]
                    asin = instance_info['asin']
                    if asin in instances:
                        # occurance
                        instances[asin]['repeat'] = instances[asin]['repeat'] + 1
                        # quantity
                        instances[asin]['quantity'] = instances[asin]['quantity'] + instance_info['quantity']
                        instances[asin]['bin_list'].append(i)
                    else:
                        instances[asin]={}
                        instances[asin]['repeat'] = 1
                        instances[asin]['quantity'] = instance_info['quantity']
                        instances[asin]['name'] = instance_info['name']
                        bin_list = []
                        bin_list.append(i)
                        instances[asin]['bin_list'] = bin_list
    return instances


In [ ]:
metadata,n_images = get_metadata(img_dir, meta_dir)

In [ ]:
print(len(metadata))

In [ ]:
instances = get_instance_data(metadata)

In [ ]:
print("dumping metadata.json...")
with open('metadata.json','w') as fp:
    json.dump(metadata,fp)
print("dumping instances.json...")
with open('instances.json','w') as fp:
    json.dump(instances,fp)

In [ ]:
import os
from os import listdir
import os.path
import numpy as np
import random

img_dir= "/kaggle/input/amazonimage/IMAGEDATA/"
meta_dir = "/kaggle/input/amazon-bucket/METADATA"

#img_list = listdir(img_dir)
#N = len(img_list)
N = 535234
list_random = list(range(N))
random.shuffle(list_random)

# finding images that metadata exists
meta_avail = np.zeros(N, dtype=bool)
for i in range(N):
    meta_fname = os.path.join(meta_dir,('%05d.json'%(i+1)))
    if os.path.isfile(meta_fname):
        meta_avail[i] = True

# assign validataion set
valset = np.zeros(N, dtype=bool)
n_valset = int(round(N*0.1))
count = 0
random.shuffle(list_random)
for i in range(N):
    idx = list_random[i]
    if meta_avail[idx]:
        valset[idx]=True
        count = count + 1
        if count == n_valset:
            break

In [ ]:
train_f = open('random_train.txt','w')
val_f = open('random_val.txt','w')
for i in range(N):
    if meta_avail[i]:
        if valset[i]:
            val_f.write("%d\n" % (i+1))
        else:
            train_f.write("%d\n" % (i+1))

In [ ]:
import os
import json
import unicodedata
import numpy as np
import json
import random

# loading metaata
random_train = "random_train.txt"
random_val = "random_val.txt"
metadata_file = "metadata.json"

hard = False

def get_quantity(idx):
    quantity = 0
    if metadata[idx]:
        quantity = metadata[idx]['EXPECTED_QUANTITY']
    return quantity

def get_moderate_list(split_file):
    print("loading random split file")
    train_list = []
    with open(split_file) as f:
        for line in f.readlines():
            idx = int(line)-1
            quantity = get_quantity(idx)
            if hard:
                train_list.append([idx,quantity])
            else:
                if quantity < 6:
                    train_list.append([idx,quantity])
    return train_list 


In [ ]:
print("loading metadata!")
with open(metadata_file) as json_file:
    metadata = json.load(json_file)
N = len(metadata)

train_list = get_moderate_list(random_train)
val_list = get_moderate_list(random_val)

In [ ]:
print("dumping train and val sets into json file")
if hard:
    out_fname = 'counting_train_hard.json'
else:
    out_fname = 'counting_train.json'
with open(out_fname,'w') as f:
    json.dump(train_list,f)

if hard:
    out_fname = 'counting_val_hard.json'
else:
    out_fname = 'counting_val.json'
with open(out_fname,'w') as f:
    json.dump(val_list,f)

In [ ]:
os.listdir()

In [ ]:
import torch.utils.data as data
import torch
#return img.resize((200,200),Image.BILINEAR)
#        if os.path.exists(os.path.join(self.root, img_path))==False:
#            img_path = "237454.jpg"
from PIL import Image
import numpy as np
import os
import os.path
import json

def default_loader(path):
    img = Image.open(path).convert('RGB')
    return img.resize((200,200),Image.BILINEAR)

class ImageFolderCounting(data.Dataset):
    def __init__(self, root, list_file, transform=None, target_transform=None,
                 loader=default_loader):
        print('loading data_list json file...')
        with open(list_file) as f:
            data_list = json.loads(f.read())
        print('finished loading data_list json file')
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.data_list = data_list
        self.N = len(self.data_list)
    
    def __getitem__(self, index):
        # pick up the image
        item = self.data_list[index]
        target = int(item[1])
        img_path = '%05d.jpg' % (item[0]+1)
        if os.path.exists(os.path.join(self.root, img_path))==False:
            img_path = "237454.jpg"
        img = self.loader(os.path.join(self.root, img_path))
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target
    
    def __len__(self):
        return self.N

In [ ]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import numpy as np
from PIL import Image

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))


parser = argparse.ArgumentParser(description='PyTorch ABID Counting')
parser.add_argument('data', metavar='DIR', help='path to dataset')
parser.add_argument('--arch', '-a', metavar='ARCH', default='resnet34', choices=model_names, help='model architecture: ' + ' | '.join(model_names) + ' (default: resnet34)')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N', help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=10, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N', help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch_size', default=128, type=int, metavar='N', help='mini-batch size (default: 256)')
parser.add_argument('--lr', '--learning-rate', default=0.1, type=float, metavar='LR', help='initial learning rate')
parser.add_argument('--lrd','--learning-rate-decay-step', default=10, type=int, metavar='N', help='learning rate decay epoch')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M', help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float, metavar='W', help='weight decay (default: 1e-4)')
parser.add_argument('--print-freq', '-p', default=10, type=int, metavar='N', help='print frequency (default: 10)')
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--evaluate', default=False, type=bool, metavar='BOOL', help='evaluate or train')

def show_image(im,target):
    print('count: %d' % target[0])
    npimg = np.transpose(im[0].mul(255).byte().numpy(), (1,2,0))
    img_batch = npimg
    for idx in range(5):
        print('count : %d' % target[idx+1])
        npimg = np.transpose(im[idx+1].mul(255).byte().numpy(), (1,2,0))
        img_batch = np.concatenate((img_batch,npimg), axis=0)
    img_pair_disp = Image.fromarray(img_batch)
    img_pair_disp.show()

best_prec = 0
train_loss_list = []
val_acc_list= []

def main():
    global args, best_prec, train_loss_list, val_acc_list
    #args = parser.parse_args(args=[])
    args, unknown = parser.parse_known_args()
    # create model
    print("=> creating model '{}'".format(args.arch))
    net = models.__dict__[args.arch]()

    in_features = net.fc.in_features
    new_fc = nn.Linear(in_features,6)
    net.fc = new_fc

    #net.cuda()

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            net.load_state_dict(checkpoint['state_dict'])
            args.start_epoch = checkpoint['epoch']
            best_prec = checkpoint['best_prec']
            train_loss_list = checkpoint['train_loss_list']
            val_acc_list = checkpoint['val_acc_list']
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
    net.to(DEVICE)
    params = net.parameters()
    snapshot_fname = "snapshots/%s.pth.tar" % args.arch
    snapshot_best_fname = "snapshots/%s_best.pth.tar" % args.arch

    cudnn.benchmark = True

    # Data loading code
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_loader = torch.utils.data.DataLoader(
        ImageFolderCounting("/kaggle/input/amazonimage/IMAGEDATA/", 'counting_train.json', transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        ImageFolderCounting("/kaggle/input/amazonimage/IMAGEDATA/", 'counting_val.json', transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(params, args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # evaluate on validation set
    if args.evaluate == True:
        validate(val_loader, net, criterion, True)
        return
    
    for epoch in range(args.start_epoch, args.epochs):
        # train for one epoch

        train(train_loader, net, criterion, optimizer, epoch)
        # evaluate on validation set
        prec = validate(val_loader, net, criterion, True)

        # remember best prec@1 and save checkpoint
        is_best = prec > best_prec
        best_prec = max(prec, best_prec)
    return net
    

def train(train_loader, net, criterion, optimizer, epoch):
    cur_lr = adjust_learning_rate(optimizer, epoch)

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    train_acc = AverageMeter()

    # switch to train mode
    net.train()

    end = time.time()

    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        target = target.cuda()
        input = input.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = net(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        score,idx = torch.max(output.data,1)
        correct = (target==idx)
        acc = float(correct.sum())/input.size(0)

        losses.update(loss.data, input.size(0))
        train_acc.update(acc, input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if i%100==0:
            print('Epoch: [{0}][{1}/{2}] lr {cur_lr:.5f}\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {train_acc.val:.3f} ({train_acc.avg:.3f})'.format(
                   epoch, i, len(train_loader), cur_lr=cur_lr, batch_time=batch_time,
                   data_time=data_time, loss=losses, train_acc=train_acc))
        #train_loss_list.append(losses.val)

def validate(val_loader, net, criterion, file_out):
    batch_time = AverageMeter()
    losses = AverageMeter()
    val_acc = AverageMeter()

    # switch to evaluate mode
    net.eval()
    
    if file_out==True:
        f = open('counting_result.txt','w') 

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input = input.cuda()
            input_var = torch.autograd.Variable(input, volatile=True)
            target_var = torch.autograd.Variable(target, volatile=True)

            # compute output
            output = net(input_var)
            loss = criterion(output, target_var)

            # measure accuracy and record loss
            score,idx = torch.max(output.data,1)
            correct = (target==idx)
            acc = float(correct.sum())/input.size(0)
 
            if file_out==True:
                for j in range(input.size(0)):
                    f.write('%d\n' % idx[j].item())

            # measure accuracy and record loss
            losses.update(loss.data, input.size(0))
            val_acc.update(acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            if i%100==0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec {val_acc.val:.3f} ({val_acc.avg:.3f})'.format(
                       i, len(val_loader), batch_time=batch_time, loss=losses,
                       val_acc=val_acc))

    print(' * Prec {val_acc.avg:.3f}'.format(val_acc=val_acc))
    if file_out==True:
        f.close()

    val_acc_list.append(val_acc.avg)
    return val_acc.avg


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 1/10 every args.lrd epochs"""
    lr = args.lr * (0.1 ** (epoch // args.lrd))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr


In [ ]:
net = main()

In [ ]:
import json
import numpy as np

with open('/kaggle/working/counting_val.json') as f:        
    val_list = json.loads(f.read())

n = 0
perclass_correct = np.zeros(6) 
perclass_dist = np.zeros(6) 
perclass_N = np.zeros(6)

confusionmatrix = np.zeros((6,6))
with open('counting_result.txt') as f:
    for line in f:
        pred = int(line)
        gt = int(val_list[n][1])
        perclass_correct[gt] = perclass_correct[gt] + int(pred==gt) 
        perclass_dist[gt] = perclass_dist[gt] + np.power(pred-gt,2)
        perclass_N[gt] = perclass_N[gt] + 1
        n = n+1
        confusionmatrix[gt][pred] = confusionmatrix[gt][pred]+1

print('accuracy')
print('%d/%d (%f)' %(perclass_correct.sum(), perclass_N.sum(), perclass_correct.sum()/perclass_N.sum()))
print('RMSE(Root mean squared error)')
print(np.sqrt(perclass_dist.sum()/perclass_N.sum()))
print('Per class accuracy')
print(perclass_correct/perclass_N)
print('Per class RMSE')
print(np.sqrt(perclass_dist/perclass_N))

In [ ]:
print(confusionmatrix)

In [ ]:
print(perclass_correct,perclass_N)